In [4]:
#take picture from video stream on pressing escape key
#pictures are taken with 2 top horizontal light strips and the table light towards the administration door (with blinders closed)
import cv2

cv2.namedWindow("tripod cam")
vc = cv2.VideoCapture(1)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    cv2.imshow("tripod cam", frame)
    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break

cv2.imwrite('Images/testresult1.png', frame)
vc.release()
cv2.destroyWindow("tripod cam")

In [ ]:
#live filter application (1 mask at a time)

import cv2
import numpy as np

# define the list of boundaries
#we need red white and black
#https://www.etutorialspoint.com/index.php/329-detect-specific-color-from-image-using-python-opencv
boundaries = [#BGR
	([165, 135, 70], [185, 160, 120]), #blue pills
]

cv2.namedWindow("tripod cam")
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    rval, img = vc.read()
else:
    rval = False

while rval:
    rval, img = vc.read()
    key = cv2.waitKey(20)

    # loop over the boundaries
    for (lower, upper) in boundaries:
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        # find the colors within the specified boundaries and apply
        # the mask
        mask = cv2.inRange(img, lower, upper)
        output = cv2.bitwise_and(img, img, mask = mask)
        # show the images
        cv2.imshow("images", np.hstack([img, output]))
        key = cv2.waitKey(20)

    if key == 27: # exit on ESC
        break

vc.release()
cv2.destroyWindow("images")
cv2.destroyWindow("tripod cam")

In [4]:
import cv2
import numpy as np

img = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 1\\Python OpenCV2\\Images\\testresult3.png")

# define the list of boundaries
boundaries = [ #BGR lower limit - upper limit
	([80, 75, 20], [140, 125, 60]), #blue
	([0, 40, 140], [10, 85, 210]), #orange
	([20, 100, 120], [80, 165, 185]) #yellow
]

# loop over the boundaries
i = 0
colour_list = ['blue', 'orange', 'yellow']

for (lower, upper) in boundaries:
    # create NumPy arrays from the boundaries
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    # find the colors within the specified boundaries and apply
    # the mask

    # Gaussian blur: Kernel size: 15x15 Sigma=2
    blur = cv2.GaussianBlur(img, (15, 15), 2) #blur image https://stackoverflow.com/questions/45943602/removing-pixels-less-than-n-sizenoise-in-an-image-open-cv-python

    # Apply color mask to have different images for each color
    mask = cv2.inRange(blur, lower, upper)

    output = cv2.bitwise_and(img, img, mask = mask)
    kernel = np.ones((3,3), dtype = np.uint8)

    # Erosion to avoid noise connected components
    output = cv2.erode(output, kernel) #to get rid of the yellow spots on the silver parts

    # Denoising using Non-local Means
    output = cv2.fastNlMeansDenoisingColored(output) #get rid of noise https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_photo/py_non_local_means/py_non_local_means.html
    
    # counting the contours and thresholding them to count the pills
    # in order to do so, the image needs to be thresholded (binary) first
    output_bw = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(output_bw,127,255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(output_bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Take into account only connected components bigger than 1500 pixels (Avoid counting noise elements)
    threshold_area = 1500
    pill_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if(area > threshold_area):
            pill_count += 1

    # show the images
    cv2.imshow("%s pill count: %d" %(colour_list[i], pill_count), output) #np.hstack([img, output])
    cv2.imwrite('Images/testresult3_output_img%i.png'%i, output)
    i += 1
    cv2.waitKey(0)